# LangChain中的Agent实现

这个笔记本展示了如何在LangChain中构建和使用Agent（智能代理）。Agent是能够利用工具执行复杂任务的自主系统，可以根据用户输入动态选择使用什么工具以及如何使用这些工具。

## 主要内容

本教程涵盖了以下内容：

1. **基本Agent的创建与使用**
   - 使用OpenAI函数调用API创建Agent
   - 配置Agent执行器处理工具交互

2. **自定义工具开发**
   - 基于本地文档的检索工具
   - 接入博查(Bocha)搜索API的网络搜索工具

3. **知识库集成**
   - 使用FAISS向量存储创建本地知识库
   - 将文档集成到检索工具中供Agent使用

4. **错误处理与调试**
   - 处理SSL连接问题
   - 修复JSON解析错误
   - 安全加载本地持久化存储

这个笔记本提供了完整的代码实现，演示了如何让大语言模型(LLM)与外部工具和数据源进行交互，创建能够回答问题、搜索信息和处理用户请求的智能系统。

## 适用场景

- 构建能够检索内部知识库的问答系统
- 开发可以进行网络搜索的信息助手
- 设计能够进行推理和工具选择的智能代理

通过本教程，您将学习如何结合LangChain的Agent框架和外部API，构建功能强大的AI应用程序。

In [5]:
import os
import requests
from typing import Optional, Type, ClassVar
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import create_openai_functions_agent,AgentExecutor
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.document_loaders import DirectoryLoader,TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain import hub


# 初始化大模型
llm = ChatOpenAI(
    model = "qwen-turbo",
    api_key = os.getenv("API_KEY"),
    base_url = os.getenv("API_BASEURL")
)

# 制作一个本地知识库查询的工具

# 初始化加载器,仅加载txt文件
loader = DirectoryLoader(
    "D:\\AI_Project\\LangChainTutorials\\LangChain\\DB", 
    glob="**/*.txt",
    loader_cls=TextLoader
)

# 加载文档
documents = loader.load()

# 初始化文档分割器
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100,
    length_function = len
)

# 分割文档
splits_docs = splitter.split_documents(documents)

# 初始化文本嵌入模型
embeddings = DashScopeEmbeddings(
    model="text-embedding-v2",
    dashscope_api_key=os.getenv("API_KEY")
)

# 初始化向量数据库
vector_store = FAISS.from_documents(
    documents= splits_docs,
    embedding= embeddings
)

# 初始化检索器
retriever = vector_store.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k" : 2}
)

# 创建检索器工具
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="retriever",
    description="两本书，一本是关于理财知识的书籍。一本是关于犯罪的小说。"
)

# 制作一个使用博查API的网络搜索工具
# 定义博查搜索工具输入
class BochaSearchInput(BaseModel):
    query: str = Field(..., description="搜索查询")
    
# 定义博查搜索工具
class BochaSearchTool(BaseTool):
    name: ClassVar[str] = "bocha_search"  # 使用ClassVar提供类型注解
    description: ClassVar[str] = "使用博查搜索引擎查询最新信息。适用于需要实时网络信息的查询。"
    args_schema: ClassVar[Type[BaseModel]] = BochaSearchInput
    
    def _run(self, query: str) -> str:
        api_key = os.getenv("BOCHA_API_KEY")
        if not api_key:
            return "错误: 未设置博查API密钥"
        
        url = "https://api.bocha.io/search"  # 根据实际API调整
        
        headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        
        data = {
            "query": query,
        }
        
        try:
            # 增加超时和重试设置
            from requests.adapters import HTTPAdapter
            from urllib3.util.retry import Retry
        
            session = requests.Session()
            retry = Retry(
                total=5,
                backoff_factor=0.5,
                status_forcelist=[500, 502, 503, 504]
            )
            adapter = HTTPAdapter(max_retries=retry)
            session.mount('https://', adapter)
            
            # 使用会话对象而不是直接调用requests.post
            response = session.post(
                url, 
                headers=headers, 
                json=data,
                timeout=30  # 增加超时时间
            )
            response.raise_for_status()
            results = response.json()
            
            formatted_results = "搜索结果:\n\n"
            for i, result in enumerate(results.get("results", []), 1):
                title = result.get("title", "无标题")
                url = result.get("url", "无链接")
                snippet = result.get("snippet", "无摘要")
                
                formatted_results += f"{i}. {title}\n"
                formatted_results += f"   链接: {url}\n"
                formatted_results += f"   摘要: {snippet}\n\n"
            
            return formatted_results
        except Exception as e:
            return f"搜索时出错: {str(e)}"

# 创建搜索工具
bocha_tool = BochaSearchTool()

# 创建工具集合
tools = [bocha_tool,retriever_tool]

# 从hub里获取提示词
prompt = hub.pull("hwchase17/openai-functions-agent")

# 创建智能体
agent = create_openai_functions_agent(llm,tools,prompt)

# 创建智能体执行器
agent_executor = AgentExecutor(
    agent = agent,
    tools = tools,
    verbose= True
)

# 不调用任何工具，直接访问大模型回答
for chunk in agent_executor.stream({"input": "你好"}):
    print(chunk, end="", flush=True)

# Agent调用检索器工具
for chunk in agent_executor.stream({"input": "如何变得有钱？"}):
    print(chunk, end="", flush=True)
    
# # 调用web查询工具，需要充值博查API账户，暂时无法使用，返回403错误信息
# for chunk in agent_executor.stream({"input": "阿里巴巴2024年的ESG报告"}):
#     print(chunk, end="", flush=True)

d:\Anaconda\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new None chain...
你好！有什么我可以帮助你的？

> Finished chain.
{'output': '你好！有什么我可以帮助你的？', 'messages': [AIMessage(content='你好！有什么我可以帮助你的？', additional_kwargs={}, response_metadata={})]}

> Entering new None chain...
{'actions': [AgentActionMessageLog(tool='retriever', tool_input={'query': '如何变得有钱'}, log="\nInvoking: `retriever` with `{'query': '如何变得有钱'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{"query": "如何变得有钱"}', 'name': 'retriever'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'qwen-turbo'}, id='run-cbd1d821-5a2c-40e3-9764-5019d1401d0e')])], 'messages': [AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{"query": "如何变得有钱"}', 'name': 'retriever'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'qwen-turbo'}, id='run-cbd1d821-5a2c-40e3-9764-5019d1401d0e')]}
Invoking: `retriever` with `{'query': '如何变得有钱'}`


大多数人没有意识到在生活中，不在于你挣了多少钱而在于你留下了多少钱。我们都听说过，一个穷人